<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#Notebook-workflow" data-toc-modified-id="Notebook-workflow-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Notebook workflow</a></span></li><li><span><a href="#Search-criteria" data-toc-modified-id="Search-criteria-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Search criteria</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Data</a></span><ul class="toc-item"><li><span><a href="#Data-dictionary" data-toc-modified-id="Data-dictionary-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Data dictionary</a></span></li></ul></li><li><span><a href="#References" data-toc-modified-id="References-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>References</a></span></li></ul></li><li><span><a href="#Establish-programming-components" data-toc-modified-id="Establish-programming-components-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Establish programming components</a></span><ul class="toc-item"><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Import libraries</a></span></li><li><span><a href="#Establish-parameters" data-toc-modified-id="Establish-parameters-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Establish parameters</a></span></li><li><span><a href="#Define-functions" data-toc-modified-id="Define-functions-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Define functions</a></span></li></ul></li><li><span><a href="#Twitter-search-API-wrapper---searchtweets" data-toc-modified-id="Twitter-search-API-wrapper---searchtweets-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Twitter search API wrapper - searchtweets</a></span><ul class="toc-item"><li><span><a href="#Load-credentials" data-toc-modified-id="Load-credentials-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Load credentials</a></span></li><li><span><a href="#Call-the-API" data-toc-modified-id="Call-the-API-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Call the API</a></span></li></ul></li><li><span><a href="#Review-and-clean-the-data" data-toc-modified-id="Review-and-clean-the-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Review and clean the data</a></span><ul class="toc-item"><li><span><a href="#Create-some-new-columns" data-toc-modified-id="Create-some-new-columns-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Create some new columns</a></span></li></ul></li><li><span><a href="#Prepare-data-to-write-to-the-SQL-database" data-toc-modified-id="Prepare-data-to-write-to-the-SQL-database-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Prepare data to write to the SQL database</a></span></li><li><span><a href="#Check-that-the-data-made-it-to-the-database" data-toc-modified-id="Check-that-the-data-made-it-to-the-database-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Check that the data made it to the database</a></span></li></ul></div>

# Overview

Tweet data from Twitter were downloaded for the purpose of understanding information about the fire as it progressed.  To do this, the tool used was the Twitter Premium search Application Programming Interface (API) which is a subscription based service in which data charges apply.  Therefore, careful consideration was given to pulling only the most relevant tweets.

## Notebook workflow

This notebook's logic follows these steps:

1. Call the twitter API to retrieve tweet data
    * Determine search criteria
    * Establish a start and end to a search time window
    * Call the API to return tweets for every 12-hour period in the time window
    * Store the return values for each API in a DataFrame and append this to as list of dataframe
    * Write a file to disk containing the data for this call
2. Concatenate the DataFrames in step 1 to a single DataFrame
3. Clean the data
    * Derive values from the profile_location attribute
    * Clean the all_text attribute and create a new column
    * Assign the time_window values to each tweet
4. Write a subset and the cleaned data to an SQL database


## Search criteria

After qualitatively examining the returned values from several queries, it was determined that the following search criteria would be used:

* Keywords: campfire
* Language: English
* Profile Region: California
* Excluding retweets

The API calls ran in 12 hour segments starting at November 08, 2018 02:00 UTC and ending November 26, 2018 02:00 UTC corresponding to November 07, 2018 18:00 US/Pacific time to November 25, 2018 18:00 US/Pacific time.  During each interval a maximum of 500 tweets would be returned.

## Data

33 attributes were returned for each tweet including such elements as the text of the tweet, the creation time, id, name, profile location, follower and following counts.  For the purposes of this analysis, a subset of attributes were selected and these were added to some derived values.   In this notebook, these are the values uploaded to the SQL database using sqlalchemy.  The following is a data dictionary of these fields.

### Data dictionary


|field                 | Description             |
|----------------------|-------------------------|
|all_text| The complete text of the tweet including retweet information|
|created_at_datetime| The creation time of the tweet|
|hashtags| The hash tags in the tweet| 
|id | The tweet's identification label| 
|name| The name of the sender|    
|follower_count| The follower count|
|following_count| The following count|
|tweet_type| The tweet type, original, quote, retweet|
|user_entered_text| The text entered by the sender|
|user_id| The user ID|
|prof_region| The profile region derived from the profile_location attribute|
|prof_subregion| The profile subregion derived from the profile_location attribute|
|prof_locality| The profile locality derived from the profile_location attribute|
|prof_long| The profile longitude derived from the profile_location attribute|
|prof_lat| The profile latitude derived from the profile_location attribute|
|clean_text| The all_text field after cleaning to remove URLs and punctuation|
|time_window| A label corresponding to the first update report that would have this information|
|retweet_count| The retweet count|

The prof_region, prof_subregion, prof_locality, prof_long, prof_lat, clean_text, time_window variables are all derived from the data returned with each tweet.  The time_window variable deserves further explanation.  Since fire progress reports were made available each day at 07:00 and 19:00 local time, we assigned a flag to each tweet to identify the first progress report that would have had the information in the tweet. 



## References
- https://github.com/ryanmcgrath/twython
- https://github.com/twitterdev/tweet_parser
- https://github.com/twitterdev/search-tweets-python
- https://pymotw.com/2/datetime/index.html#module-datetime
- https://developer.twitter.com/en/docs/tweets/search/guides/integrating-premium.html
- https://stackoverflow.com/questions/10997577/python-timezone-conversion/18646797#18646797


# Establish programming components

## Import libraries

In [1]:
#!pip install pip twython
#!pip install searchtweets
#!pip install tweet_parser

import os
from searchtweets import collect_results
from searchtweets import ResultStream, gen_rule_payload, load_credentials
import pandas as pd
import numpy as np
from time import sleep
from datetime import datetime, timedelta
from sqlalchemy import create_engine
import psycopg2
import regex as re
import pytz


/Users/stephengodfrey/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Establish parameters

In [2]:
# Twitter API access keys
API_KEY = <'API-Key'>
API_SECRET_KEY = <'API_SECRET_KEY'>
ACCESS_TOKEN = <'ACCESS_TOKEN'>
ACCESS_TOKEN_SECRET = <'ACCESS_TOKEN_SECRET'>
endpoint = 'https://api.twitter.com/1.1/tweets/search/fullarchive/T2P4Dev.json'
BEARER_TOKEN = <'BEARER_TOKEN'>

# Environment variables for the search_tweets API
os.environ['SEARCHTWEETS_ENDPOINT'] = endpoint
# export SEARCHTWEETS_USERNAME=
# export SEARCHTWEETS_PASSWORD=
os.environ['SEARCHTWEETS_BEARER_TOKEN'] = BEARER_TOKEN
os.environ['SEARCHTWEETS_ACCOUNT_TYPE']= 'premium'
os.environ['SEARCHTWEETS_CONSUMER_KEY']= API_KEY
os.environ['SEARCHTWEETS_CONSUMER_SECRET']= API_SECRET_KEY

# Data path for the csv files
data_path = "/Users/stephengodfrey/Documents/SteveSchool/dsi/projects/project_4/Team_2_Project_4/data/twitter"

# Database engine
conn_str = <'Postgres connection string'>
engine = create_engine(conn_str)


## Define functions

In [4]:
# Advance a date time interval
def advance_time(start_time = "2018-11-08 02:00", int = 24*60):
    # convert to date time format and interval
    time = datetime.strptime(start_time, '%Y-%m-%d %H:%M') + timedelta(minutes = int)
    # return time converted back to a string
    return time.strftime('%Y-%m-%d %H:%M')
    

# Return a dictionary of tweet elements
def create_tweet_dict(tweet = None):

    # create a new dictionary
    t_dict = {}
      
    # add tweet elements
    t_dict['id'] = tweet.id
    t_dict['created_at_seconds'] = tweet.created_at_seconds
    t_dict['created_at_datetime'] = tweet.created_at_datetime
    t_dict['created_at_string'] = tweet.created_at_string
    t_dict['user_id'] = tweet.user_id
    t_dict['screen_name'] = tweet.screen_name
    t_dict['name'] = tweet.name
    t_dict['bio'] = tweet.bio
    t_dict['follower_count'] = tweet.follower_count
    t_dict['following_count'] = tweet.following_count
    t_dict['text'] = tweet.text
    t_dict['tweet_type'] = tweet.tweet_type
    t_dict['user_entered_text'] = tweet.user_entered_text
    t_dict['lang'] = tweet.lang
    t_dict['poll_options'] = tweet.poll_options
    t_dict['quote_or_rt_text'] = tweet.quote_or_rt_text
    t_dict['all_text'] = tweet.all_text
    t_dict['geo_coordinates'] = tweet.geo_coordinates
    t_dict['profile_location'] = tweet.profile_location
    t_dict['tweet_links'] = tweet.tweet_links
    t_dict['most_unrolled_urls'] = tweet.most_unrolled_urls
    t_dict['user_mentions'] = tweet.user_mentions
    t_dict['hashtags'] = tweet.hashtags
    t_dict['quoted_tweet'] = tweet.quoted_tweet
    t_dict['retweeted_tweet'] = tweet.retweeted_tweet
    t_dict['embedded_tweet'] = tweet.embedded_tweet
    t_dict['gnip_matching_rules'] = tweet.gnip_matching_rules
    t_dict['generator'] = tweet.generator
    t_dict['in_reply_to_screen_name'] = tweet.in_reply_to_screen_name
    t_dict['in_reply_to_user_id'] = tweet.in_reply_to_user_id
    t_dict['in_reply_to_status_id'] = tweet.in_reply_to_status_id
    t_dict['favorite_count'] = tweet.favorite_count
    t_dict['quote_count'] = tweet.quote_count
    t_dict['retweet_count'] = tweet.retweet_count
    
    # return the dictionatry
    return t_dict

# Write the posts data to a SQL table
def write_to_database(df, engine = None, table = None):
    # write posts to the posts table
    if engine == None:
        return
    else:
#         df.to_sql(table, con=engine, index=False, if_exists='append')
        df.to_sql(table, con=engine, index=False, if_exists='append')


# Read in data from SQL database
def read_from_database(SQL, engine = None):
    # write posts to the posts table
    if engine == None:
        return
    return pd.read_sql(SQL, con = engine)


# Pull tweets over the specified time window
def pull_tweets_time_window(topics = 'CampFire', 
                            start_time = "2018-11-08 02:00", 
                            end_time = "2018-11-09 02:00",
                            interval = 12*60, 
                            data_path = "~/"):
    
    # Variable definition
    pause_time = 1 #second
    cnt = 1
    
    # Set the beginning and first interval
    int_start = start_time
    int_end = advance_time(start_time = int_start, 
                           int = interval)
    
    # list of dataframes
    list_df = []

    while datetime.strptime(int_end, '%Y-%m-%d %H:%M') <= datetime.strptime(end_time, '%Y-%m-%d %H:%M'):
   
        # Tweet list
        t_list = []
        
        # Generate the search rule
        rule = gen_rule_payload(topics, from_date = int_start, 
                                to_date = int_end, 
                                results_per_call = 500)
                
        
        # ask if user wants to continue
        if cnt > 40:
            cont = input('Iteration %d: Rule %s \n Do you want to continue (y or n)?' % (cnt, str(rule)))
            if cont.upper()[:1] != 'Y':
                print('Exiting search')
                break
        
        # Return the search stream
        rs = ResultStream(rule_payload=rule,
                          max_results=500,
                          max_pages=500,
                          **premium_search_args)
        
        # Add each tweet as a dictionary to a list
        t_list = [create_tweet_dict(tweet) for tweet in list(rs.stream())]
        
        # Convert t_dict to a DataFrame
        df = pd.DataFrame(t_list)
        
        # Write to a csv file
        out_file = "tweets_" + int_start.replace(' ','_') + '_'+ int_end.replace(' ','_')
        print(os.path.join(data_path, out_file))
        df.to_csv(os.path.join(data_path, out_file))
        
        # Append to a list of data frames
        list_df.append(df)
        
        # Advance interval
        int_start = int_end
        int_end = advance_time(start_time = int_start, 
                               int = interval)
        cnt += 1
    
    return list_df

# Function to get location details
def get_profile_locations(p_loc = {}):
    # establish variables
    region = np.NaN
    sub_region = np.NaN
    locality = np.NaN
    long = np.NaN
    lat = np.NaN
  
    # Try to assign values for each of these variables
    try:
        region = p_loc['region']
    except:
        pass
    try:
        sub_region = p_loc['sub_region']
    except:
        pass
    try:
        locality = p_loc['locality']
    except:
        pass
    try:
        long = p_loc['geo']['coordinates'][0]
    except:
        pass
    try:
        lat = p_loc['geo']['coordinates'][1]
    except:
        pass

    # Return the profile location details        
    return region, sub_region, locality, long, lat

    
# Function to read through
def assign_profile_location_details(df = None, column = 'profile_location'):
    # Check to see if df 
    if not isinstance(df, pd.DataFrame):
        raise Exception('The df parameter must be a pandas.DatFrame')

    for i in df.index:
        df.loc[i, 'prof_region'], df.loc[i,'prof_subregion'], df.loc[i,'prof_locality'], \
        df.loc[i, 'prof_long'], df.loc[i,'prof_lat']\
        = get_profile_locations(df.loc[i,column])
        
        
# Create a function to scrub a text string
def scrub_text(in_text):
    # Remove urls, numbers and punctuation
    word_list = re.sub(r'(http)\S*|[^a-zA-Z 0-9\n]','',in_text.lower()).split()
    return ' '.join(word_list)


# Code to convert times
# https://stackoverflow.com/questions/10997577/python-timezone-conversion/18646797#18646797
def convert_datetime_timezone(dt, tz1, tz2):
    # Get timezone objects
    tz1 = pytz.timezone(tz1)
    tz2 = pytz.timezone(tz2)

    # Localize the datetime object
    dt = tz1.localize(dt)
    dt = dt.astimezone(tz2)
    # Convert the output format by creating string then reformatting
    dt = dt.strftime("%Y-%m-%d %H:%M:%S")
    dt = datetime.strptime(dt, "%Y-%m-%d %H:%M:%S")
    
    return dt

# assign a time window based on the time stamp of the tweet
def assign_time_window(dt):
    # convert UTC to california time
    dt = convert_datetime_timezone(dt,'UTC', 'US/Pacific')
    
    # strip out the date
    date = dt.strftime('%Y-%m-%d')    
    cut_1 = datetime.strptime(date + " 07:00:00", "%Y-%m-%d %H:%M:%S")
    cut_2 = datetime.strptime(date + " 19:00:00", "%Y-%m-%d %H:%M:%S")
    
    # Check which window this falls into and return proper window
    if dt < cut_1:
        return cut_1.strftime("%Y-%m-%d %H:%M:%S")
    elif dt >= cut_1 and dt < cut_2:
        return cut_2.strftime("%Y-%m-%d %H:%M:%S")
    else:
        dt_1 = dt + timedelta(days=1)
        date_1 = dt_1.strftime('%Y-%m-%d')
        cut_3 = datetime.strptime(date_1 + " 07:00:00", "%Y-%m-%d %H:%M:%S")
        return cut_3.strftime("%Y-%m-%d %H:%M:%S")
    
    

# Twitter search API wrapper - searchtweets

## Load credentials 

In [17]:
premium_search_args = load_credentials("test",
                                       yaml_key="search_tweets_premium",
                                       env_overwrite=False)


# premium_search_args = load_credentials("~/.twitter_keys.yaml",
#                                        yaml_key="search_tweets_premium",
#                                        env_overwrite=False)



cannot read file test
Error parsing YAML file; searching for valid environment variables


## Call the API

In [18]:
# pull the tweets for this window - create a list of dataframes
# topics = 'fire lang:en point_radius:[-121.633759 39.767380 25mi]'
dfs =  pull_tweets_time_window(topics = 'campfire lang:en profile_region:California -is:retweet', 
                               start_time = "2018-11-10 02:00", 
                               end_time = "2018-11-26 02:00",
                               interval = 12*60,
                               data_path = data_path)


/Users/stephengodfrey/Documents/SteveSchool/dsi/projects/project_4/Team_2_Project_4/data/twitter/tweets_2018-11-10_02:00_2018-11-10_14:00
/Users/stephengodfrey/Documents/SteveSchool/dsi/projects/project_4/Team_2_Project_4/data/twitter/tweets_2018-11-10_14:00_2018-11-11_02:00
/Users/stephengodfrey/Documents/SteveSchool/dsi/projects/project_4/Team_2_Project_4/data/twitter/tweets_2018-11-11_02:00_2018-11-11_14:00
/Users/stephengodfrey/Documents/SteveSchool/dsi/projects/project_4/Team_2_Project_4/data/twitter/tweets_2018-11-11_14:00_2018-11-12_02:00
/Users/stephengodfrey/Documents/SteveSchool/dsi/projects/project_4/Team_2_Project_4/data/twitter/tweets_2018-11-12_02:00_2018-11-12_14:00
/Users/stephengodfrey/Documents/SteveSchool/dsi/projects/project_4/Team_2_Project_4/data/twitter/tweets_2018-11-12_14:00_2018-11-13_02:00
/Users/stephengodfrey/Documents/SteveSchool/dsi/projects/project_4/Team_2_Project_4/data/twitter/tweets_2018-11-13_02:00_2018-11-13_14:00
/Users/stephengodfrey/Documents/St

# Review and clean the data

In [19]:
# create a single data frame from the list of dataframes
df = pd.concat(dfs, ignore_index = True)
print(df.shape)
df

df.tail(50)



(14003, 34)


,all_text,bio,created_at_datetime,created_at_seconds,created_at_string,embedded_tweet,favorite_count,follower_count,following_count,generator,...,quoted_tweet,retweet_count,retweeted_tweet,screen_name,text,tweet_links,tweet_type,user_entered_text,user_id,user_mentions
13953,"85 souls lost, and hundreds still missing, but...","Founder, Savvy Millennial| @ForbesUnder30|Boar...",2018-11-25 15:55:35,1543161335,2018-11-25T15:55:35.000Z,{'created_at': 'Sun Nov 25 15:02:52 +0000 2018...,4,5020,2724,"{'link': 'http://twitter.com/download/iphone',...",...,{'created_at': 'Sun Nov 25 15:02:52 +0000 2018...,0,None,SavIsSavvy,"85 souls lost, and hundreds still missing, but...","[{'url': 'https://t.co/R6A8dgt34a', 'expanded_...",quote,"85 souls lost, and hundreds still missing, but...",47824384,[]
13954,"#CampFire is now 100% contained, good job fire...","Enthusiastic world explorer, digital creative,...",2018-11-25 15:54:15,1543161255,2018-11-25T15:54:15.000Z,{'created_at': 'Sun Nov 25 15:02:52 +0000 2018...,5,6590,2424,"{'link': 'http://twitter.com/download/iphone',...",...,{'created_at': 'Sun Nov 25 15:02:52 +0000 2018...,0,None,Natbat,"#CampFire is now 100% contained, good job fire...","[{'url': 'https://t.co/kNMnQVGCUn', 'expanded_...",quote,"#CampFire is now 100% contained, good job fire...",12161,[]
13955,Big news: the #CampFire is 100% contained!!!!!...,2004 and 2008 Olympic distance swimmer 🇸🇻 Open...,2018-11-25 15:54:14,1543161254,2018-11-25T15:54:14.000Z,None,3,129,421,"{'link': 'http://twitter.com/download/iphone',...",...,None,0,None,goldafishm,Big news: the #CampFire is 100% contained!!!!!...,[],tweet,Big news: the #CampFire is 100% contained!!!!!...,3103609124,[]
13956,@The_Embarcadero @Deplora26124312 @ButteSherif...,Founder of http://Worldie.com 🌎 http://Motiva...,2018-11-25 15:54:11,1543161251,2018-11-25T15:54:11.000Z,None,1,3060,4170,"{'link': 'https://mobile.twitter.com', 'name':...",...,None,1,None,XTinaTaft,@The_Embarcadero @Deplora26124312 @ButteSherif...,"[{'url': 'https://t.co/mkIsqTyb2y', 'expanded_...",tweet,@The_Embarcadero @Deplora26124312 @ButteSherif...,847762217546137600,"[{'screen_name': 'The_Embarcadero', 'name': 'T..."
13957,"Some GOOD NEWS, the #CampFire is 100% CONTAINE...",Oroville News Live has the latest news in the ...,2018-11-25 15:50:43,1543161043,2018-11-25T15:50:43.000Z,None,3,547,463,"{'link': 'http://www.facebook.com/twitter', 'n...",...,None,0,None,OrovilleNews1,"Some GOOD NEWS, the #CampFire is 100% CONTAINE...","[{'url': 'https://t.co/ob00tk6si3', 'expanded_...",tweet,"Some GOOD NEWS, the #CampFire is 100% CONTAINE...",816441139867750401,[]
13958,We served many but most importantly we are bri...,Chef~Founder of Plated Events. Food First Resp...,2018-11-25 15:49:51,1543160991,2018-11-25T15:49:51.000Z,None,6,575,755,"{'link': 'http://twitter.com/download/iphone',...",...,None,0,None,chefjasoncollis,We served many but most importantly we are bri...,"[{'url': 'https://t.co/nL6wFbw1Pu', 'expanded_...",tweet,We served many but most importantly we are bri...,59894087,[]
13959,@CALFIRE_ButteCo @ButteSheriff @paradise_ca @C...,"Multimedia Writer/Editor 🌊🌲🌳🏔️Oceans, mountain...",2018-11-25 15:49:06,1543160946,2018-11-25T15:49:06.000Z,None,2,15167,7580,"{'link': 'http://twitter.com', 'name': 'Twitte...",...,None,4,None,EdJoyce,@CALFIRE_ButteCo @ButteSheriff @paradise_ca @C...,"[{'url': 'https://t.co/R2iESCG6mI', 'expanded_...",tweet,@CALFIRE_ButteCo @ButteSheriff @paradise_ca @C...,73452506,"[{'screen_name': 'CALFIRE_ButteCo', 'name': 'C..."
13960,"#CampFire 153,336 acres &amp; 100% contained\n...",Skywarn Spotter #BU36 & CoCoRahs Observer CA-B...,2018-11-25 15:48:51,1543160931,2018-11-25T15:48:51.000Z,None,12,1205,92,"{'link': 'http://twitter.com', 'name': 'Twitte...",...,None,4,None,ButteWxSpotter,"#CampFire 153,336 acres &amp; 100% contained\n...","[{'url': 'https://t.co/PNjYWykQWf', 'expanded_...",tweet,"#CampFire 153,336 acres &amp; 100% contained\n...",135319086,[]
13961,#BREAKING: #CampFire now 

In [20]:
df.columns

Index(['all_text', 'bio', 'created_at_datetime', 'created_at_seconds',
       'created_at_string', 'embedded_tweet', 'favorite_count',
       'follower_count', 'following_count', 'generator', 'geo_coordinates',
       'gnip_matching_rules', 'hashtags', 'id', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'lang',
       'most_unrolled_urls', 'name', 'poll_options', 'profile_location',
       'quote_count', 'quote_or_rt_text', 'quoted_tweet', 'retweet_count',
       'retweeted_tweet', 'screen_name', 'text', 'tweet_links', 'tweet_type',
       'user_entered_text', 'user_id', 'user_mentions'],
      dtype='object')

## Create some new columns

In [21]:
# Steps to clean the tweet data
# get location details if it exists
assign_profile_location_details(df = df, column = 'profile_location')
df['clean_text'] = df['all_text'].apply(scrub_text)
df['time_window'] = df['created_at_datetime'].apply(assign_time_window)


In [22]:
# Examine the columns of the DataFrame
df.columns


Index(['all_text', 'bio', 'created_at_datetime', 'created_at_seconds',
       'created_at_string', 'embedded_tweet', 'favorite_count',
       'follower_count', 'following_count', 'generator', 'geo_coordinates',
       'gnip_matching_rules', 'hashtags', 'id', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'lang',
       'most_unrolled_urls', 'name', 'poll_options', 'profile_location',
       'quote_count', 'quote_or_rt_text', 'quoted_tweet', 'retweet_count',
       'retweeted_tweet', 'screen_name', 'text', 'tweet_links', 'tweet_type',
       'user_entered_text', 'user_id', 'user_mentions', 'prof_region',
       'prof_subregion', 'prof_locality', 'prof_long', 'prof_lat',
       'clean_text', 'time_window'],
      dtype='object')

# Prepare data to write to the SQL database

In [24]:
# Create a new DataFrame that has columns to write to database
write_features = ['all_text','created_at_datetime', 'hashtags', 'id', 'name',
                 'follower_count', 'following_count','tweet_type','user_entered_text', 
                  'user_id', 'prof_region', 'prof_subregion', 'prof_locality', 
                  'prof_long', 'prof_lat', 'clean_text', 'time_window', 'retweet_count'
                 ]

df_to_write = df[write_features]


In [25]:
# Write these data to SQL on AWS
write_to_database(df_to_write, engine = engine, table = 'twitter')


# Check that the data made it to the database

In [26]:
SQL = "SELECT * FROM twitter"
df_in = read_from_database(SQL, engine = engine)


In [29]:
print(df_in.shape)
df_in.tail(20)


(15513, 18)


,all_text,created_at_datetime,hashtags,id,name,follower_count,following_count,tweet_type,user_entered_text,user_id,prof_region,prof_subregion,prof_locality,prof_long,prof_lat,clean_text,time_window,retweet_count
15493,Camp Fire Update 11.25.18 AM:\n#CampFire #Butt...,2018-11-25 15:26:02,"{CampFire,ButteStrong,ParadiseStrong,ButteReco...",1066714634546163712,CAL FIRE Butte Unit/Butte County Fire Department,29732,189,tweet,Camp Fire Update 11.25.18 AM:\n#CampFire #Butt...,1711024998,California,None,None,-119.75126,37.25022,camp fire update 112518 am campfire buttestron...,2018-11-25 19:00:00,100
15494,🚨🚨EXCELLENT NEWS! The #CampFire has been decla...,2018-11-25 15:24:35,{CampFire},1066714273173258240,CHP Oroville,635,32,quote,🚨🚨EXCELLENT NEWS! The #CampFire has been decla...,1035296345081360385,California,Butte County,Oroville,-121.55636,39.51378,excellent news the campfire has been declared ...,2018-11-25 19:00:00,3
15495,#campfire the most destructive fire in Califor...,2018-11-25 15:22:32,{campfire},1066713757030670336,Stacy Chen,450,821,quote,#campfire the most destructive fire in Califor...,2827184059,California,Los Angeles County,Los Angeles,-118.24368,34.05223,campfire the most destructive fire in californ...,2018-11-25 19:00:00,0
15496,JUST IN: #CampFire is 100% contained https://t...,2018-11-25 15:19:31,{CampFire},1066712996599091200,KRON4 News,202222,3217,tweet,JUST IN: #CampFire is 100% contained https://t...,19031057,California,San Francisco County,San Francisco,-122.41942,37.77493,just in campfire is 100 contained,2018-11-25 19:00:00,108
15497,"#CampFire 153,336 acres - 100% contained. Than...",2018-11-25 15:19:23,{CampFire},1066712960892993536,Chuck Reynolds,5308,352,quote,"#CampFire 153,336 acres - 100% contained. Than...",7779312,None,None,None,NaN,NaN,campfire 153336 acres 100 contained thank you ...,2018-11-25 19:00:00,0
15498,Thank you firefighters 🙏🏻 #CampFire #ButteCoun...,2018-11-25 15:19:09,"{CampFire,ButteCountyFires}",1066712901753270273,LetsDoItForJumbo,75,549,quote,Thank you firefighters 🙏🏻 #CampFire #ButteCoun...,283711431,California,None,None,-119.75126,37.25022,thank you firefighters campfire buttecountyfir...,2018-11-25 19:00:00,0
15499,Cal Fire says the #CampFire is 100% contained....,2018-11-25 15:18:53,{CampFire},1066712835349082113,Angela Corral,394,784,tweet,Cal Fire says the #CampFire is 100% contained....,1952548614,California,San Francisco County,San Francisco,-122.41942,37.77493,cal fire says the campfire is 100 contained kq...,2018-11-25 19:00:00,6
15500,#BREAKING: 100 percent containment reached in ...,2018-11-25 15:12:42,"{BREAKING,CampFire,ButteCounty}",1066711280377942016,ABC7 News,540473,25182,tweet,#BREAKING: 100 percent containment reached in ...,18993395,California,San Francisco County,San Francisco,-122.41942,37.77493,breaking 100 percent containment reached in ca...,2018-11-25 19:00:00,136
15501,#RT @CAL_FIRE: #CampFire [final] Pulga Road at...,2018-11-25 15:10:21,"{RT,CampFire}",1066710690986164225,Jonathan Cox,504,637,tweet,#RT @CAL_FIRE: #CampFire [final] Pulga Road at...,785868908955709440,California,San Francisco County,San Francisco,-122.41942,37.77493,rt calfire campfire final pulga road at camp c...,2018-11-25 19:00:00,0
15502,"RT CAL_FIRE ""#CampFire [final] Pulga Road at C...",2018-11-25 15:06:26,{CampFire},1066709701826621440,CALIFORNIA PIO,170,11,tweet,"RT CAL_FIRE ""#CampFire [final] Pulga Road at C...",4726597453,California,None,None,-119.75126,37.25022,rt calfire campfire final pulga road at camp c...,2018-11-25 19:00:00,0
